<a href="https://colab.research.google.com/github/w-okada/asrclient/blob/master/w_okada's_ASR_Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### w-okada's Automated Speech Recognition Client  | **Google Colab**

## READ ME - VERY IMPORTANT
This is an attempt to run [Text To Speech Client](https://github.com/w-okada/asr-client) on Google Colab, still not perfect but is totally usable.

---

###Always use Colab GPU (**VERY VERY VERY IMPORTANT!**)
You need to use a Colab GPU so the Voice Changer can work faster and better\
Use the menu above and click on **Runtime** » **Change runtime** » **Hardware acceleration** to select a GPU (**T4 is the free one**)



In [1]:
#=================Updated=================
# @title **[1]** Clone repository and install dependencies
# @markdown This first step will download the latest version of Voice Changer and install the dependencies. **It can take some time to complete.(~5min)**

#@markdown ---
# @markdown By using Google Drive, you can avoid re-downloading already downloaded versions. "/content/drive/MyDrive/asrclient" is used as working directory.

import os
import time
import subprocess
import threading
import shutil
import base64
import codecs
import torch
import sys

from typing import Literal, TypeAlias

%cd /content


# Configs
Run_Cell=0
GOOGLE_DRIVE_WORKDIR = "/content/drive/MyDrive/asr-client"
Use_Drive=False #@param {type:"boolean"}
Initialize_Working_Directory=False #@param {type:"boolean"}


current_version_hash=None
latest_version_hash=None

# Check GPU
if torch.cuda.is_available():
    print("GPU is available")
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available")
    # sys.exit("No GPU available. Change runtime.")


notebook_env=0
if os.path.exists('/content'):
  notebook_env=1
  print("Welcome to Colab Mode")
  from google.colab import drive

elif os.path.exists('/kaggle/working'):
  notebook_env=2
  print("Welcome to Kaggle Mod")
else:
  notebook_env=3
  print("Welcome!")

from IPython.display import clear_output, Javascript

if notebook_env==1 and Use_Drive==True:
  work_dir = GOOGLE_DRIVE_WORKDIR
  if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

  if Initialize_Working_Directory:
    user_input = input(f"Are you sure you want to delete the working directory '{GOOGLE_DRIVE_WORKDIR}'? (y/n): ").strip().lower()
    if user_input == 'y':
        print(f"Deleting {GOOGLE_DRIVE_WORKDIR}")
        !rm -r {GOOGLE_DRIVE_WORKDIR}
    else:
        print("Skipping deletion of working directory.")


  if not os.path.exists(work_dir):
    !mkdir -p {work_dir}

  print("Checking latest version...")
  if os.path.exists(f'{work_dir}/latest_hash.txt'):
    current_version_hash = open(f'{work_dir}/latest_hash.txt').read().strip()
  else:
    current_version_hash = None

  !curl -s -L https://huggingface.co/wok000/asrclient000_colab/resolve/main/latest_hash.txt -o latest_hash.txt
  latest_version_hash = open('latest_hash.txt').read().strip()

  print(f"current_version_hash: {current_version_hash}")
  print(f"latest_version_hash : {latest_version_hash}")

  if current_version_hash != latest_version_hash:
    print(f"hash not match -> download latest version")

    latest_hash_path=f'{work_dir}/latest_hash.txt'

    !curl -L https://huggingface.co/wok000/asrclient000_colab/resolve/main/asrclient_latest_for_colab -o {work_dir}/asrclient_latest_for_colab
    !curl -L https://huggingface.co/wok000/asrclient000_colab/resolve/main/web_front_latest.zip -o {work_dir}/web_front_latest.zip

    !cp latest_hash.txt {latest_hash_path}
    print("Download is done.")
  else:
    print("hash matched. skip download")

else:
  work_dir = "/content"
  print("Downloading the latest asrclient... ")
  !curl -s -L https://huggingface.co/wok000/asrclient000_colab/resolve/main/latest_hash.txt -o latest_hash.txt
  latest_version_hash = open('latest_hash.txt').read().strip()

  !curl -L https://huggingface.co/wok000/asrclient000_colab/resolve/main/asrclient_latest_for_colab -o {work_dir}/asrclient_latest_for_colab
  !curl -L https://huggingface.co/wok000/asrclient000_colab/resolve/main/web_front_latest.zip -o {work_dir}/web_front_latest.zip

  print("Download is done.")


#### MOVE TO WORK DIR ###
%cd {work_dir}
!chmod 0700 asrclient_latest_for_colab

print("Installing modules... ",end="")
!sudo apt-get install -y libportaudio2 > /dev/null 2>&1
!pip install  pyngrok > /dev/null 2>&1
print("Install is done.")

Run_Cell=1

!unzip -qq -o  web_front_latest.zip

/content
GPU is available
GPU Name: Tesla T4
Welcome to Colab Mode
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1151  100  1151    0     0   3409      0 --:--:-- --:--:-- --:--:--  3405
100 1028M  100 1028M    0     0  40.2M      0  0:00:25  0:00:25 --:--:-- 39.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1217  100  1217    0     0   4797      0 --:--:-- --:--:-- --:--:--  4810
100  432k  100  432k    0     0   697k      0 --:--:-- --:--:-- --:--:-- 16.5M
Download is done.
/content
Installing modules... Install is done.


In [2]:
# @title **[2]** Start server (~ 3min)
# @markdown This cell will start the server, the first time that you run it will download the models, so it can take a while (2~4 minutes)

# @markdown If you want to use ngrok, please input your token in the option section below. If you encounter a 403 error with the colab proxy, using ngrok can sometimes help to work around it.
# @markdown https://dashboard.ngrok.com/

# @markdown And if you cannot use ngrok, you can use the cell client (experimental). Run the cell below.


# @markdown ### Options:
ClearConsole = True  # @param {type:"boolean"}
Play_Notification = True  # @param {type:"boolean"}
NgrokToken = ""  # @param {type:"string"}

PORT=8010
NGROK_URL_FILE = "ngrok_url.txt"

LOG_FILE = f"/content/LOG_FILE_{PORT}.log"

from IPython.display import Audio, display
def play_notification_sound(url):
    display(Audio(url=url, autoplay=True))

from google.colab.output import eval_js




if NgrokToken =="":
  get_ipython().system_raw(f'LD_LIBRARY_PATH=/usr/lib64-nvidia:/usr/lib/x86_64-linux-gnu ./asrclient_latest_for_colab cui --port {PORT} --no_cui true --https False >{LOG_FILE} 2>&1 &')
else:
  # get_ipython().system_raw(f'LD_LIBRARY_PATH=/usr/lib64-nvidia:/usr/lib/x86_64-linux-gnu ./asrclient_latest_for_colab cui --port {PORT} --no_cui true --https False --ngrok_token {NgrokToken} --ngrok_proxy_url_file {NGROK_URL_FILE} >{LOG_FILE} 2>&1 &')
  get_ipython().system_raw(f'./asrclient_latest_for_colab cui --port {PORT} --no_cui true --https False --ngrok_token {NgrokToken} --ngrok_proxy_url_file {NGROK_URL_FILE} >{LOG_FILE} 2>&1 &')


import socket
def wait_for_server():
  elapsed_time = 0
  start_time = time.time()


  while True:
      time.sleep(1)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', PORT))
      if result == 0:
          break
      sock.close()
      # 時刻を出力
      current_time = time.time()
      elapsed_time = int(current_time - start_time)
      clear_output(wait=True)
      print(f"Waiting for server... elapsed: {elapsed_time}sec")
      try:
        with open(LOG_FILE, 'r') as f:
            lines = f.readlines()[-5:]
            for line in lines:
                print(line.strip())
      except:
        pass

  if ClearConsole:
      clear_output()
  print("--------- SERVER READY! ---------")
  print(f"Your server is available. elapsed: {elapsed_time}sec")
  proxy = eval_js( "google.colab.kernel.proxyPort(" + str(PORT) + ")" )
  print(f"colab proxy: {proxy}")
  if NgrokToken != "":
    with open(NGROK_URL_FILE, "r") as f:
      ngrok_url = f.read().strip()
    print(f"Ngrok URL: {ngrok_url}")
  print("---------------------------------")
  if Play_Notification==True:
    play_notification_sound('https://huggingface.co/wok000/voices/resolve/main/the_application_is_ready.wav')
wait_for_server()


--------- SERVER READY! ---------
Your server is available. elapsed: 30sec
colab proxy: https://yleyaxd6e7-496ff2e9c6d22116-8010-colab.googleusercontent.com/
---------------------------------


In [3]:
# @title **[3]** Start Cell Client (Experimental)
# @markdown Launch the client within the cell. This can be used if proxy or ngrok cannot be used.If the behavior is unstable, re-running the cell may resolve the issue.

from IPython.display import HTML, SVG,Javascript
from IPython.core.magic import register_cell_magic

def dojs3():
  return HTML(f"""
<script>var colab_server_port={PORT}</script>
<script>var colab_server=1</script>
<script defer="defer" src="http://localhost:{PORT}/index.js"></script>
<div id="app" style="width:100%;height:100%"></div>
  """)
dojs3()